In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras import Model


2023-12-05 20:14:43.533697: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 20:14:43.533740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 20:14:43.534859: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 20:14:44.587053: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set the input size based on the pre-trained model you choose
input_size = (224, 224)

# Define data directories
train_dir = './jitropha/'

# Define the batch size
batch_size = 8


In [3]:
# Use data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
)


In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 26 images belonging to 2 classes.


In [5]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create the model by adding custom classification layers
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)


In [6]:
model = Model(base_model.input, x)


In [7]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [10]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10
)


Epoch 1/10
3/3 [==============================] - 1s 220ms/step - loss: 0.6194 - accuracy: 0.6111
Epoch 2/10
3/3 [==============================] - 0s 108ms/step - loss: 0.5098 - accuracy: 0.8889
Epoch 3/10
3/3 [==============================] - 0s 105ms/step - loss: 0.4353 - accuracy: 0.8333
Epoch 4/10
3/3 [==============================] - 0s 103ms/step - loss: 0.3547 - accuracy: 0.9444
Epoch 5/10
3/3 [==============================] - 0s 96ms/step - loss: 0.4549 - accuracy: 0.8889
Epoch 6/10
3/3 [==============================] - 0s 197ms/step - loss: 0.3425 - accuracy: 0.8889
Epoch 7/10
3/3 [==============================] - 0s 185ms/step - loss: 0.3271 - accuracy: 0.8889
Epoch 8/10
3/3 [==============================] - 0s 194ms/step - loss: 0.2061 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 1s 99ms/step - loss: 0.2401 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 94ms/step - loss: 0.1668 - accuracy: 1.0000


In [11]:
model.evaluate(train_generator)


4/4 [==============================] - 2s 123ms/step - loss: 0.1629 - accuracy: 1.0000


[0.16289140284061432, 1.0]

In [13]:
model.save('./Training/ripe_model/model.keras')
